In [ ]:
from gs_quant.session import Environment, GsSession
from gs_quant.instrument import FXDoubleOneTouch
from gs_quant.markets.portfolio import Portfolio
from datetime import date
import pandas as pd

In [ ]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

#### Basic Details

In [ ]:
# get list of properties of an fx knockout - Note the knockout instrument will also do RKO (Reverse Knockout)
FXDoubleOneTouch.properties()

In [ ]:
# in this example we will construct and price a portfolio of FX DNTs
fx_dbl_touches = Portfolio()

# you don't need to specify any parameters to get a valid trade.  All properties have defaults
# Note that touch_or_no_touch defaults to "No Touch", so the default trade is a DNT not a DOT (see below cell)
fx_dbl_touches.append(FXDoubleOneTouch())

#### Double **No** Touch vs Double **One** touch
`touch_or_no_touch` controls whether the option pays if either barrier_level is breached (called a Double One Touch or DOT for short).  
Or whether the option only pays if neither barrier_level has been breached by the `expiration_date` (called a Double No Touch or DNT).

In [ ]:
from gs_quant.common import TouchNoTouch
fx_dbl_touches.append(FXDoubleOneTouch(lower_barrier_level=.9, upper_barrier_level=1.2, touch_or_no_touch=TouchNoTouch.No_Touch))
fx_dbl_touches.append(FXDoubleOneTouch(lower_barrier_level=.9, upper_barrier_level=1.2, touch_or_no_touch=TouchNoTouch.Touch))

#### Buying vs Selling

In [ ]:
# buy_sell indicates whether the option is bought (long) or sold (short) 
# It can be represented by the BuySell enum or a string 
fx_dbl_touches.append(FXDoubleOneTouch(buy_sell='Buy'))

from gs_quant.common import BuySell
fx_dbl_touches.append(FXDoubleOneTouch(buy_sell=BuySell.Sell))

#### Currencies involved
`pair` is the `FXDoubleOneTouch`'s underlying currency pair. It is a string of two ccy iso codes, optionally separated with a space (' ').  
The first currency is the base (transaction) currency and the second is the quote currency.
The upper and lower barrier levels are then levels corresponding to this FX rate
The option also has a `notional_currency` and `notional_amount` field, this corresponds to the conditional option payout.

In [ ]:
from gs_quant.common import Currency
# In this case, base currency is 'EUR' and quote currency is 'USD'
# This option gives the purchasor a 100k EUR payout if the barrier levels are not breached
fx_dbl_touches.append(FXDoubleOneTouch(buy_sell=BuySell.Buy, pair='EUR USD', notional_currency=Currency.EUR, notional_amount=100e3))

# Here, base currency is 'GBP' and quote currency is 'USD'
# This option gives the purchasor a 100k USD payout if either barrier is breached
fx_dbl_touches.append(FXDoubleOneTouch(buy_sell=BuySell.Buy, touch_or_no_touch="Touch", pair='GBPUSD', notional_currency=Currency.EUR, notional_amount="100k"))

#### Barrier Levels
`upper_barrier_level` and `lower_barrier_level` are the exchange rates stuck for the DOT/DNT's underlying currency pair, which can be specified by a double 
or a string with a keyword/letter. If a string is used, it represents a relative value. When the trade is resolved, we solve for the strike_price 

The specific solver keys are: 
* 'S'    - current spot rate
* 'F'    - forward
* 'D'    - Delta Strikes of a vanilla option
* 'P'    - Premium

You can use these keys with the following formats: 
* For S, F, ATM, ATMF: 's*1.05', 'F+10%', '1.05*ATMF+.01'
* For Delta Strikes, specify the option delta: '25D', '20D-.01', etc.
* You can also solve for Premium: P=<target premium>, P=<target premium> P=,<target>%, PPct=<target>

To have the upper barrier 2% above the current spot either of the following syntaxs would be valid:  
`upper_barrier_level="s+2%"` or `upper_barrier_level="s*1.02"`

In [ ]:
# Here the option is set to have fixed barrier levels 
fx_dbl_touches.append(FXDoubleOneTouch(pair='EURGBP', expiration_date="1m", lower_barrier_level=0.8, upper_barrier_level=0.95))

# The option is sold have the lower barrier 1% below spot and upper barrier 1% above the fwd
fx_dbl_touches.append(FXDoubleOneTouch(pair='EURGBP', expiration_date="1m", lower_barrier_level="s-1%", upper_barrier_level="f+1%"))

#### Option Premium
`premium_currency` is the premium amount's denominated currency.  
It can be a Currency enum or a string. By default, `premium_currency` will match the `notional_currency`.  
The `premium` will **default to the fair premium** s.t. that the PresentValue of the option + premium = 0.  
You therefore may find it helpful to set the premium explictly to 0 s.t. the PresentValue of the DOT/DNT is just the value of the option.  
`premium_payment_date` is the date the premium is exchanged. It can either be a date or string such as "fwd" or "spot"

In [ ]:
# In this case, the payout and premium currency of the EURGBP DNt will be EUR
fx_dbl_touches.append(FXDoubleOneTouch(pair='EUR GBP', notional_currency='EUR', premium_currency='EUR'))

# Here we explictly set the premium to 0 so measures like DollarPrice or PresentValue only include the option value
fx_dbl_touches.append(FXDoubleOneTouch(pair='EUR GBP', premium=0, notional_currency='EUR'))

# Here we set it the option to have forward premium by setting `premium_payment_date` to 'fwd' or 'forward'
fx_dbl_touches.append(FXDoubleOneTouch(pair='EUR GBP', premium_payment_date='fwd'))

#### Premium Solving
It is possible to solve for the level of a single barrier to reach a certain Premium (in % or absolute value)  
You can also sovle both barrier levels for a target premium in which case they will be symmetrical spaced from current spot.  
Common syntax for solving for permium in % or absolute level include `p=10%` and `p=-200k`

In [ ]:
import gs_quant.risk as risk
# e.g. Solve X in a "1m USDJPY spot-3%/X DNT" s.t. that the fair premium is 10% of the notional
upper_solve = FXDoubleOneTouch(pair='USD JPY', expiration_date='1m', lower_barrier_level='spot-3%', upper_barrier_level="p=10%")
calced = upper_solve.calc((risk.FairPremium, risk.ResolvedInstrumentValues))
print(calced[risk.FairPremium], calced[risk.ResolvedInstrumentValues].upper_barrier_level)

In [ ]:
# e.g. Solve for a 1m USDJPY DNT with symmetric strikes aorund spot s.t. the premium is 5%
double_solve = FXDoubleOneTouch(pair='USD JPY', expiration_date='1m', lower_barrier_level='p=5%', upper_barrier_level='p=5%', premium=0)
calced = double_solve.calc((risk.FairPremiumInPercent, risk.FXSpot, risk.ResolvedInstrumentValues))
resolved = calced[risk.ResolvedInstrumentValues]
print(calced[risk.FairPremiumInPercent]*100, resolved.lower_barrier_level, calced[risk.FXSpot], resolved.upper_barrier_level)

In [ ]:
pd.DataFrame(fx_dbl_touches.price())